In [1]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu

import genjax
from genjax import normal
from genjax import tfp_uniform
from genjax import trace
from genjax import gen
from genjax.inference import smc
from genjax import index_choice_map, choice_map
from genjax import diff, NoChange, UnknownChange
from genjax import index_select

console = genjax.pretty()
key = jax.random.PRNGKey(314159)

In [4]:
@genjax.gen
def f1():
    return genjax.tfp_normal(0.0, 1.0) @ "y"


@genjax.gen
def f2():
    return genjax.tfp_normal(0.0, 2.0) @ "x"


s = genjax.SwitchCombinator([f1, f2])

s_keys = jax.random.split(jax.random.PRNGKey(17), 3)
# (Just select 0 in all branches for simplicity:)
_, tr = jax.vmap(s.simulate, in_axes=(0, None))(s_keys, (1,))

In [5]:
tr["y"].mask

Array([False, False, False], dtype=bool)